# LLM Iteration 9

**Aim**

Create a Scenario class that holds default values and creates the distributions for sampling.

**Prompt**

The code below has been designed to model a stroke ward based on the following brief:

Code a discrete-event simulation model of a stroke ward in a hospital in python 3.10 and simpy 4.
Please code the full model specified. Do not return a simplified version. Show all code. 

All time units in the model are in days.
Each patient in the model has a unique identifier. The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.

Patients to arrive in a stroke ward that admits four different patient types: Stroke, TIA, Complex Neurological or Other. Each patient type had a different interarrival time distribution and each patient type arrival is independent of the other patient types. There are 10 beds on the stroke ward. The patient should only be admitted if there is an available bed. The patient holds the bed until their length of stay has elapsed. After the length of stay has elapsed, the patient leaves the model. 

All patient types must have their own generator function. 

The model should print out useful information after each event. 
The model should include a user settable run length. This defaults to 2 months. 

The interarrival time distribution and parameters of patients are dependent on the patient type. This describes the average time between required admissions. For each distribution time is measured in days. 
Stroke = Exponential: 1.2
TIA = Exponential: 9.3
Complex Neurological = Exponential: 3.6
Other = Exponential: 3.2
Numpy should be used for sampling. Each interarrival distribution should have its own numpy.random.Generator object. 

The length of stay distribution and parameters of patients are dependent on the patient type. For each distribution time is measured in days. 
Stroke = Lognormal: mean 7.4, standard deviation 8.6
TIA = Lognormal: mean 1.8, standard deviation 2.3
Complex Neurological = Lognormal: mean 4.0, standard deviation 5.0
Other = Lognormal: mean 3.8, standard deviation 5.2
The inputs for the lognormal distribution are mu and sigma and should be calculated as follows, where v is the standard deviation squared and m is the mean:
phi = math.sqrt(v + m^2)
mu = math.log(m^2/phi)
sigma = math.sqrt(math.log(phi^2/m^2))
Numpy should be used for sampling. Each length of stay distribution should have its own numpy.random.Generator object. Each random number generator should use the random seed specified for each patient type. 

```python
import simpy
import numpy as np
import math

class StrokeWard:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=NUM_BEDS)
        self.patient_count = 0
        self.wait_times = []

        # Initialize patient generators for each patient type
        env.process(self.patient_generator(INTERARRIVAL_TIMES['Stroke'], 'Stroke', LENGTH_OF_STAY_PARAMS['Stroke'], random_seed=42))
        env.process(self.patient_generator(INTERARRIVAL_TIMES['TIA'], 'TIA', LENGTH_OF_STAY_PARAMS['TIA'], random_seed=10))
        env.process(self.patient_generator(INTERARRIVAL_TIMES['Complex Neurological'], 'Complex Neurological', LENGTH_OF_STAY_PARAMS['Complex Neurological'], random_seed=20))
        env.process(self.patient_generator(INTERARRIVAL_TIMES['Other'], 'Other', LENGTH_OF_STAY_PARAMS['Other'], random_seed=30))

    def admit_patient(self, patient):
        arrival_time = self.env.now
        with self.beds.request() as bed:
            yield bed
            wait_time = self.env.now - arrival_time
            self.wait_times.append(wait_time)
            print(f"Patient {patient.id} of type {patient.type} admitted at day {round(self.env.now, 2)}")
            yield self.env.timeout(patient.length_of_stay)
            print(f"Patient {patient.id} of type {patient.type} discharged at day {round(self.env.now, 2)}")

    def patient_generator(self, interarrival_time, patient_type, length_of_stay_params, random_seed=None):
        rng_interarrival = np.random.default_rng(random_seed)
        rng_length_of_stay = np.random.default_rng(random_seed)

        while True:
            yield self.env.timeout(rng_interarrival.exponential(interarrival_time))

            mu, sigma = calculate_lognormal_params(*length_of_stay_params)
            length_of_stay_sample = rng_length_of_stay.lognormal(mu, sigma)

            patient = Patient(self.patient_count, patient_type, length_of_stay_sample)
            self.patient_count += 1
            self.env.process(self.admit_patient(patient)) 

class Patient:
    def __init__(self, id, type, length_of_stay):
        self.id = id
        self.type = type
        self.length_of_stay = length_of_stay

def calculate_lognormal_params(mean, std_dev):
    phi = math.sqrt(std_dev**2 + mean**2)
    mu = math.log((mean**2) / phi)
    sigma = math.sqrt(math.log((phi**2) / (mean**2)))
    return mu, sigma

# Define the parameters
NUM_BEDS = 10
RUN_LENGTH = 60  # Default run length in days (2 months)
INTERARRIVAL_TIMES = {
    'Stroke': 1.2,
    'TIA': 9.3,
    'Complex NL': 3.6,
    'Other': 3.2
}
LENGTH_OF_STAY_PARAMS = {
    'Stroke': (7.4, 8.6),
    'TIA': (1.8, 2.3),
    'Complex NL': (4.0, 5.0),
    'Other': (3.8, 5.2)
}

env = simpy.Environment()
ward = StrokeWard(env)

env.run(until=RUN_LENGTH)

# Calculate and print the mean wait time for a bed at the end of the run
mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")
```

Update this code to create a Scenario class that holds the default values for the model and creates the distributions used by the model and initialises the random seeds of each.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
import simpy
import numpy as np
import math

In [2]:
class Scenario:
    def __init__(self, num_beds=10, run_length=60, interarrival_times=None,
                 length_of_stay_params=None):
        self.num_beds = num_beds
        self.run_length = run_length
        self.interarrival_times = interarrival_times if interarrival_times \
            is not None else {
            'Stroke': 1.2,
            'TIA': 9.3,
            'Complex NL': 3.6,
            'Other': 3.2
        }
        self.length_of_stay_params = length_of_stay_params if \
            length_of_stay_params is not None else {
            'Stroke': (7.4, 8.6),
            'TIA': (1.8, 2.3),
            'Complex NL': (4.0, 5.0),
            'Other': (3.8, 5.2)
        }

    def create_distributions(self):
        distributions = {}
        for patient_type, params in self.length_of_stay_params.items():
            v = params[1] ** 2
            m = params[0]
            phi = math.sqrt(v + m ** 2)
            mu = math.log((m ** 2) / phi)
            sigma = math.sqrt(math.log((phi ** 2) / (m ** 2))) # INSERTED )
            distributions[patient_type] = {'mu': mu, 'sigma': sigma}
        return distributions

    def initialize_random_seeds(self):
        random_seeds = {'Stroke': 42, 'TIA': 10, 'Complex NL': 20, 'Other': 30}
        return random_seeds

class StrokeWard:
    def __init__(self, env, scenario):
        self.env = env
        self.beds = simpy.Resource(env, capacity=scenario.num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.scenario = scenario

        # Initialize patient generators for each patient type
        distributions = scenario.create_distributions()
        random_seeds = scenario.initialize_random_seeds()

        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        self.discharge_store = []
        self.patient_store = []
        ##################################

        for patient_type in scenario.interarrival_times:           
            env.process(self.\
                        patient_generator(scenario.\
                                          interarrival_times[patient_type],
                                          patient_type,
                                          distributions[patient_type],
                                          random_seeds[patient_type]))

    def admit_patient(self, patient):
        arrival_time = self.env.now
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            self.patient_store.append(patient)
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            ##################################
            
            wait_time = self.env.now - arrival_time
            self.wait_times.append(wait_time)
            print(f'Patient {patient.id} of type {patient.type} admitted at '
                  f'day {round(self.env.now, 2)}')
            yield self.env.timeout(patient.length_of_stay)
            
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(bed_leave_time - bed_arrival_time)
            self.discharge_store.append(patient.type)
            ##################################
            print(f'Patient {patient.id} of type {patient.type} discharged at '
                  f'day {round(self.env.now, 2)}')

    def patient_generator(self, interarrival_time, patient_type,
                          length_of_stay_params, random_seed=None):
        rng_interarrival = np.random.default_rng(random_seed)
        rng_length_of_stay = np.random.default_rng(random_seed)

        while True:
            yield self.env.timeout(rng_interarrival.\
                                   exponential(interarrival_time))

            mu = length_of_stay_params['mu']
            sigma = length_of_stay_params['sigma']
            length_of_stay_sample = rng_length_of_stay.lognormal(mu, sigma)

            patient = Patient(self.patient_count, patient_type,
                              length_of_stay_sample)
            self.patient_count += 1
            self.env.process(self.admit_patient(patient)) 

class Patient:
    def __init__(self, id, type, length_of_stay):
        self.id = id
        self.type = type
        self.length_of_stay = length_of_stay

In [3]:
env = simpy.Environment()
scenario = Scenario()
ward = StrokeWard(env, scenario)

env.run(until=scenario.run_length)

# Calculate and print the mean wait time for a bed at the end of the run
mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 0 of type Complex NL discharged at day 2.54
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 6 of type Other admitted at day 4.22
Patient 6 of type Other discharged at day 5.05
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 10 of type Complex NL admitted at day 5.89
Patient 11 of type Complex NL admitted at day 6.23
Patient 8 of type Other discharged at day 7.15
Patient 12 of type Complex NL admitted at day 7.15
Patient 9 of type Stroke discharged at day 7.53
Patient 13 of type TIA admitted at day 7.53
Patient 11 of type Complex NL discharged at day 7.79
Patient 4 of type Other discharged at day 7.81
Patient 13 of type TIA discharged at d

**Checks**

In [4]:
class TestCase:
    def __init__(self, env, ward):
        self.env = env
        self.ward = ward
        self.run_length = ward.scenario.run_length

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=self.run_length)

        if self.env.now > self.run_length:
            print('\nThe simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=self.run_length)

        for bed_use in self.ward.bed_in_use_store:
            if bed_use > ward.scenario.num_beds:
                print('\nThe number of beds in use exceeded limit.')

    def test_length_of_stay_change(self, shortest_stay):
        '''
        Checks how changing the length of stay for different patinet types
        influences the patient discharges. Change the length of stay parameters
        so one patient type has a much shorter length of stay (shortest_stay).
        Checks that only a few of the other patient types are discharged and
        gives their length of stay.
        '''
        self.env.run(until=self.run_length)

        for los, type in zip(self.ward.los_store, self.ward.discharge_store):
            if type != shortest_stay:
                print(f'\nPatient type {type} stayed for {los}.')

    def test_patients(self):
        '''
        Prints out patient type and their length of stay.
        '''
        self.env.run(until=self.run_length)
        print('\n')
        for pat in self.ward.patient_store:
            print(f'{pat.id} ({pat.type}): LOS = {pat.length_of_stay}')

In [5]:
# Checks run length
env = simpy.Environment()
scenario = Scenario(run_length=10)
ward = StrokeWard(env, scenario)

test = TestCase(env, ward)
test.test_run_length()

mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 0 of type Complex NL discharged at day 2.54
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 6 of type Other admitted at day 4.22
Patient 6 of type Other discharged at day 5.05
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 10 of type Complex NL admitted at day 5.89
Patient 11 of type Complex NL admitted at day 6.23
Patient 8 of type Other discharged at day 7.15
Patient 12 of type Complex NL admitted at day 7.15
Patient 9 of type Stroke discharged at day 7.53
Patient 13 of type TIA admitted at day 7.53
Patient 11 of type Complex NL discharged at day 7.79
Patient 4 of type Other discharged at day 7.81
Patient 13 of type TIA discharged at d

In [6]:
# Checks that is only ever 1 patient if there is only 1 bed
env = simpy.Environment()
scenario = Scenario(num_beds=1)
ward = StrokeWard(env, scenario)

test = TestCase(env, ward)
test.test_bed_hold()

mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 0 of type Complex NL discharged at day 2.54
Patient 1 of type Other admitted at day 2.54
Patient 1 of type Other discharged at day 13.84
Patient 2 of type Complex NL admitted at day 13.84
Patient 2 of type Complex NL discharged at day 21.87
Patient 3 of type Stroke admitted at day 21.87
Patient 3 of type Stroke discharged at day 28.27
Patient 4 of type Other admitted at day 28.27
Patient 4 of type Other discharged at day 33.08
Patient 5 of type Complex NL admitted at day 33.08
Patient 5 of type Complex NL discharged at day 42.76
Patient 6 of type Other admitted at day 42.76
Patient 6 of type Other discharged at day 43.59
Patient 7 of type Complex NL admitted at day 43.59
Patient 7 of type Complex NL discharged at day 46.99
Patient 8 of type Other admitted at day 46.99
Patient 8 of type Other discharged at day 48.79
Patient 9 of type Stroke admitted at day 48.79
Patient 9 of type Stroke discharged at day 50.64
Patient 10 of type 

In [7]:
# Checks patient types other than the one with the shortest stay that have
# been discharged, to check this does not fall outside expected range
LENGTH_OF_STAY_PARAMS = {
    'Stroke': (1.5, 5.0),
    'TIA': (60, 5.0),
    'Complex NL': (60, 5.0),
    'Other': (60, 5.0)
}

env = simpy.Environment()
scenario = Scenario(length_of_stay_params=LENGTH_OF_STAY_PARAMS)
ward = StrokeWard(env, scenario)

test = TestCase(env, ward)
test.test_length_of_stay_change('Stroke')

mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 3 of type Stroke discharged at day 3.58
Patient 6 of type Other admitted at day 4.22
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 9 of type Stroke discharged at day 5.77
Patient 10 of type Complex NL admitted at day 5.89
Patient 11 of type Complex NL admitted at day 6.23
Patient 0 of type Complex NL discharged at day 58.8
Patient 12 of type Complex NL admitted at day 58.8
Patient 6 of type Other discharged at day 59.38
Patient 13 of type TIA admitted at day 59.38

Patient type Complex NL stayed for 58.03021423749276.

Patient type Other stayed for 55.15790472870431.

Mean wait time for a bed: 7.4

In [8]:
# Prints out patient type and their length of stay
env = simpy.Environment()
scenario = Scenario(run_length=10)
ward = StrokeWard(env, scenario)

test = TestCase(env, ward)
test.test_patients()

mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 0 of type Complex NL discharged at day 2.54
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 6 of type Other admitted at day 4.22
Patient 6 of type Other discharged at day 5.05
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 10 of type Complex NL admitted at day 5.89
Patient 11 of type Complex NL admitted at day 6.23
Patient 8 of type Other discharged at day 7.15
Patient 12 of type Complex NL admitted at day 7.15
Patient 9 of type Stroke discharged at day 7.53
Patient 13 of type TIA admitted at day 7.53
Patient 11 of type Complex NL discharged at day 7.79
Patient 4 of type Other discharged at day 7.81
Patient 13 of type TIA discharged at d

In [12]:
# Testing the Scenario class
scenario = Scenario()
dist = scenario.create_distributions()
seeds = scenario.initialize_random_seeds()
for type, vals in dist.items():
    print(f'{type}: Output => {vals}')
print('\n')
for type, vals in seeds.items():
    print(f'{type}: Seeds => {vals}')

Stroke: Output => {'mu': 1.5741402490781844, 'sigma': 0.9244887788739675}
TIA: Output => {'mu': 0.10377864905244422, 'sigma': 0.9838780573319793}
Complex NL: Output => {'mu': 0.9158026888876274, 'sigma': 0.9700429601128635}
Other: Output => {'mu': 0.8073964417840169, 'sigma': 1.0272337854143265}


Stroke: Seeds => 42
TIA: Seeds => 10
Complex NL: Seeds => 20
Other: Seeds => 30


**Reflection**

The results that were output from the model (mean wait time and patients being admitted and discharged) were the same as the previous iteration, meaning it did not change how the code was run. One missed bracket had to be inserted that was missed by the LLM. The Scenario class holds the constants and set up the default parameters but it did not initialise distributions correctly as there is no option to sample from the distributions. It only calculated the mu and sigma for the Lognormal distribution inside this class. It also initialised a single random seed inside the class, with no option to change this.